In [125]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler


Se define la funcion CargarDatos, en la cual se leen los archivos del csv, se crea la variable data, que tiene los datos de las frutas que seran usados en el clasificador mlp, y tambien las variables labels y sublabels, que son los nombres de las frutas y los subtipos de frutas

In [126]:
def CargarDatos():
  df     = pd.read_csv('combined_data.csv')
  data   = df.drop(['fruit_label','fruit_name','fruit_subtype'],axis=1)
  labels = df.fruit_name
  sublabels = df.fruit_subtype
  return(data,labels,sublabels)

Se define cada una de las variables definidas en la funcion CargarDatos

In [127]:
data, labels, sublabels = CargarDatos()
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

Se entrenan los modelos de las frutas tanto como los subtipos de estas

In [128]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.25,random_state=1)

In [129]:
subtrain_data, subtest_data, subtrain_labels, subtest_labels = train_test_split(data, sublabels, test_size=0.25,random_state=1)

Con este codigo se buscan los mejores parametros para el clasificador mlp, los valores son todos casos mas generales, sin embargo para aclarar, las capas ocultas suelen ser como maximo entre una y dos, y por la simplicidad del modelo, se dejaran asi, en caso de ser mas complejo se podrian aumentar las capas ocultas, generalmente es mejor aumentar la cantidad de capas en vez de la cantidad de neuronas. 

In [151]:
# Definir el clasificador base
mlp = MLPClassifier(max_iter=500)

# Definir el espacio de hiperparámetros
param_grid = {
    'solver': ['sgd', 'adam'],
    'alpha': [1e-5, 1e-4, 1e-3],
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
    'activation': ['relu', 'tanh', 'logistic'],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [300, 500, 1000]
}

# Ejecutar GridSearchCV
grid_search = GridSearchCV(mlp, param_grid, n_jobs=-1, cv=5)
grid_search.fit(subtrain_data, subtrain_labels)

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros:", grid_search.best_params_)

Mejores hiperparámetros: {'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001, 'max_iter': 500, 'solver': 'adam'}


c:\Users\felip\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Se definen el clasificador junto a sus parametros

In [152]:
Clasificador = MLPClassifier(solver='adam',
                    alpha=0.001,
                    hidden_layer_sizes=(50,50),
                    max_iter = 500,
                    activation ='logistic',
                    random_state=1)

In [153]:
tmp = Clasificador.fit(train_data, train_labels)

c:\Users\felip\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


se calculan las medidas de accuracy, precision, recall y f1-score para las frutas

In [154]:
Clasificador.score(train_data, train_labels)

0.7983193277310925

In [155]:
predicciones  = Clasificador.predict(test_data)
accuracy      = accuracy_score(predicciones, test_labels)
precision     = precision_score(test_labels, predicciones, average='macro')
recall        = recall_score(test_labels, predicciones, average='macro')
print("Accuracy sobre los datos de prueba: ", accuracy)
print("Precision sobre los datos de prueba: ", precision)
print("Recall sobre los datos de prueba: ", recall)

Accuracy sobre los datos de prueba:  0.875
Precision sobre los datos de prueba:  0.8875
Recall sobre los datos de prueba:  0.8870798319327732


In [147]:
subClasificador = MLPClassifier(solver='adam',
                    alpha=0.001,
                    hidden_layer_sizes=(100,),
                    max_iter = 300,
                    activation ='logistic',
                    random_state=1)

In [148]:
tmp = subClasificador.fit(subtrain_data, subtrain_labels)

c:\Users\felip\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


se calculan las medidas de accuracy, precision, recall y f1-score para los subtipos de fruta

In [149]:
subClasificador.score(subtrain_data, subtrain_labels)

0.5798319327731093

In [150]:
predicciones  = subClasificador.predict(subtest_data)
accuracy      = accuracy_score(predicciones, subtest_labels)
precision     = precision_score(subtest_labels, predicciones, average='macro')
recall        = recall_score(subtest_labels, predicciones, average='macro')
print("Accuracy sobre los datos de prueba: ", accuracy)
print("Precision sobre los datos de prueba: ", precision)
print("Recall sobre los datos de prueba: ", recall)

Accuracy sobre los datos de prueba:  0.6
Precision sobre los datos de prueba:  0.4666666666666667
Recall sobre los datos de prueba:  0.4932900432900433


c:\Users\felip\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
